In [1]:
!pip install contractions
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 14.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=16b9a487c99e0c7d09eb5142962c4ee2657abdc9687b17aeb327759b1204fb60
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [2]:
#importing libraries
import pandas as pd
from langdetect import detect
import contractions
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
wn = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
#loading the data
data = pd.read_csv('/content/drive/MyDrive/DL Projects/Classification Projects/RNN/Fake News Classification/data/train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
#checking for missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
#considering title and label fields for training and testing the model
#dropping the null values
dt = data[['title','label']]
dt = dt.dropna()
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20242 entries, 0 to 20799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20242 non-null  object
 1   label   20242 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 474.4+ KB


In [6]:
#checking the language of the news articles (in this project only eng. are considered)
def det(x):
    try:
        lang = detect(x)
    except:
        lang = 'other'
    return lang

dt['lang'] = dt['title'].apply(det)
dt.head()

,title,label,lang
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,en
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0,en
2,Why the Truth Might Get You Fired,1,en
3,15 Civilians Killed In Single US Airstrike Hav...,1,en
4,Iranian woman jailed for fictional unpublished...,1,en


In [7]:
dt.lang.value_counts()

en       19264
de         339
ru         154
es         135
fr          88
nl          26
ca          24
no          22
af          19
ar          19
it          19
sv          13
pt          12
et          12
da          10
lt          10
ro          10
id          10
tr           9
tl           9
cy           6
so           6
hr           5
vi           4
sw           3
bg           3
fi           3
pl           2
el           2
sq           1
fa           1
zh-cn        1
other        1
Name: lang, dtype: int64

In [8]:
dt = dt[dt['lang']=='en']

In [11]:
#Expanding contractions
def expand(text):
  data = []
  for word in text.split():
      data.append(contractions.fix(word))
  # print(data)
  return ' '.join(word for word in data)

In [12]:
dt['exp_title'] = dt['title'].apply(lambda x:expand(x))
dt.head()

,title,label,lang,exp_title
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,en,House Dem Aide: We Did not Even See Comey’s Le...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0,en,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired,1,en,Why the Truth Might Get You Fired
3,15 Civilians Killed In Single US Airstrike Hav...,1,en,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...,1,en,Iranian woman jailed for fictional unpublished...


In [49]:
dt['exp_title'] = dt['exp_title'].str.lower()
dt.head()

,title,label,lang,exp_title
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,en,house dem aide: we did not even see comey’s le...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0,en,"flynn: hillary clinton, big woman on campus - ..."
2,Why the Truth Might Get You Fired,1,en,why the truth might get you fired
3,15 Civilians Killed In Single US Airstrike Hav...,1,en,15 civilians killed in single us airstrike hav...
4,Iranian woman jailed for fictional unpublished...,1,en,iranian woman jailed for fictional unpublished...


In [52]:
#data preprocessing include removing stop words, converting to lower case and convert words to their lemma form
def preprocess(text):
  x = re.sub(r"[^a-z]+", ' ', text)
  x = [word for word in x.split() if not word in stopwords.words('english')]
  line = ' '.join(wn.lemmatize(word) for word in x)
  return line

In [66]:
preprocessed_text = []
for text in dt.exp_title:
  preprocessed_text.append(preprocess(text))
len(preprocessed_text)

19264

In [67]:
dt['preprocessed_text'] = preprocessed_text
dt.head()

,title,label,lang,exp_title,preprocessed_text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,en,house dem aide: we did not even see comey’s le...,house dem aide even see comey letter jason cha...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0,en,"flynn: hillary clinton, big woman on campus - ...",flynn hillary clinton big woman campus breitbart
2,Why the Truth Might Get You Fired,1,en,why the truth might get you fired,truth might get fired
3,15 Civilians Killed In Single US Airstrike Hav...,1,en,15 civilians killed in single us airstrike hav...,civilian killed single u airstrike identified
4,Iranian woman jailed for fictional unpublished...,1,en,iranian woman jailed for fictional unpublished...,iranian woman jailed fictional unpublished sto...


In [68]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19264 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   title              19264 non-null  object
 1   label              19264 non-null  int64 
 2   lang               19264 non-null  object
 3   exp_title          19264 non-null  object
 4   preprocessed_text  19264 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.4+ MB


In [69]:
#saving the data
dt.to_csv('/content/drive/MyDrive/DL Projects/Classification Projects/RNN/Fake News Classification/data/processsed.csv', index=False)